Simple Concatenation of Series and DataFrames with pd.concat

In [1]:
import pandas as pd
import numpy as np

In [2]:
#create a function to make a data frame
def makeDf(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}
    return pd.DataFrame(data, ind)

In [3]:
makeDf('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [5]:
#use pd.concat()
ser1 = pd.Series(['A','B','C'], index = [1,2,3])
ser2 = pd.Series(['D','e','f'], index = [4,5,6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    e
6    f
dtype: object

In [7]:
df1 = makeDf('AB',[1,2])
df2 = makeDf('AB',[7,9])
print(df1); print(df2);  print(pd.concat([df1, df2]))

    A   B
1  A1  B1
2  A2  B2
    A   B
7  A7  B7
9  A9  B9
    A   B
1  A1  B1
2  A2  B2
7  A7  B7
9  A9  B9


Duplicate Indices

In [10]:
x = makeDf('AB', [0,1])
y = makeDf('AB', [2,3])
y.index = x.index    #make duplicate index
print(pd.concat([x,y]))

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


In [11]:
#catching the duplicate indices error
try:
    pd.concat([x,y], verify_integrity=True)
except ValueError as e:
    print("ValueError: ",e)

ValueError:  Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


In [13]:
#ignoring indices
print(pd.concat([x,y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


### use 'join'

In [14]:
df5 = makeDf('abc', [1,2])
df6 = makeDf('bcd',[3,4])
pd.concat([df5, df6])

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,NaN,b3,c3,d3
4,NaN,b4,c4,d4


this will give some Nulll values... but we can avoid 
it by using 'join'

In [15]:
pd.concat([df5, df6], join='outer')

,a,b,c,d
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,NaN,b3,c3,d3
4,NaN,b4,c4,d4


In [16]:
pd.concat([df5,df6], join = 'inner')

,b,c
1,b1,c1
2,b2,c2
3,b3,c3
4,b4,c4


# Merge And Join


#### Relational Algebra

### Categories of join
1. one to one
2. many to one
3. many to many

In [18]:
# 1. one to one

df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})

df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
    'hire_date': [2004, 2008, 2012, 2014]})

print(df1); 
print("\n")
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR


  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [20]:
#combine
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [21]:
# 2. many to one

df4 = pd.DataFrame({'group':['Accounting', 'Engineering', 'HR'],
                   'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3)
print('\n', df4)

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

          group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve


In [22]:
pd.merge(df3,df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


In [23]:
# 3. Many to Many
df5 = pd.DataFrame({'group': ['Accounting','Accounting',
                             'Engineering', 'Engineering', 'HR', 'HR'],
                   'skills' : ['math','spreadsheet','ML','coding','spreadsheet',
                              'organization']})

In [24]:
print(df1)
print("\n", df5)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

          group        skills
0   Accounting          math
1   Accounting   spreadsheet
2  Engineering            ML
3  Engineering        coding
4           HR   spreadsheet
5           HR  organization


In [25]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheet
2,Jake,Engineering,ML
3,Jake,Engineering,coding
4,Lisa,Engineering,ML
5,Lisa,Engineering,coding
6,Sue,HR,spreadsheet
7,Sue,HR,organization


## Specification of Merge Key

Merge joins the dataframe not so nnicely
so we will use some specification

#### The on Keyword

In [31]:
print(df1)
print('\n', df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

   employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [32]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### 2. left_on and right_on

In [33]:
# """if the name of key is not same in 2 datasets
# then we can use left_on and right_on"""

'if the name of key is not same in 2 datasets\nthen we can use left_on and right_on'

In [35]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'salary': [70000,8000,20000,120000]})
df3

,name,salary
0,Bob,70000
1,Jake,8000
2,Lisa,20000
3,Sue,120000


In [37]:
pd.merge(df1, df3, left_on ='employee',right_on = 'name')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,8000
2,Lisa,Engineering,Lisa,20000
3,Sue,HR,Sue,120000
